#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [ ]:
# check the number of rows in your csv file generated by the `etl-process.py`
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

## The CSV file we generated above  titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [ ]:
from cassandra.cluster import Cluster

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
# Create a Keyspace 
try:
    session.execute("""
                            CREATE KEYSPACE IF NOT EXISTS event
                            WITH REPLICATION = 
                            {
                            'class': 'SimpleStrategy', 'replication_factor': 1
                            }
                            """)
except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('event')
except Exception as e:
    print(e)

In [ ]:
### Now we need to create tables to run the following queries. Remember, with Apache Cassandra the database tables should be model on the queries to run.

###  The First Query 1

In [ ]:
### For this query itemInSession and sessionId are considered as the COMPOSITE KEY. \
### Because we are quering the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [ ]:
# Creating the table `music_library`
query = "CREATE TABLE IF NOT EXISTS music_library"
try:
    session.execute(query + "(session_id int, item_session int, artist varchar, \
                                song text, length float, first_name varchar, last_name varchar, \
                                user_id text, \
                                PRIMARY KEY(session_id, item_session))")
except Exception as e:
    print(e)

In [ ]:
# Loop through the `event_datafile_new.csv` and extract each column into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Adding INSERT statements into the `query` variable
        query = "INSERT INTO music_library "
        query = query + "(session_id, item_session, artist, song, length, \
                            first_name, last_name, \
                            user_id) VALUES(%s, %s, %s, \
                                            %s, %s, %s, \
                                            %s, %s)"
        ## Insert each columns into the table.
        session.execute(query, (int(line[3]), int(line[8]), line[0], line[9], \
                                float(line[5]), line[1], line[4], \
                                line[10]))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## Adding SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length FROM music_library  WHERE session_id = 338 AND item_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, row.song, row.length)

### The second Query

In [ ]:
### For this query user_id, session_id, item_session are considered as the COMPOSITE KEY. \
### Beacuse we are Quering the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [ ]:
# Create the table `song_playlist_session`
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
try:
    session.execute(query + "( user_id int, session_id int, item_session int, artist varchar, \
                                first_name varchar, gender varchar, last_name varchar, \
                                length float, level varchar, location varchar, song varchar, \
                                PRIMARY KEY((user_id, session_id), item_session))")
except Exception as e:
    print(e)

                    

In [ ]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Adding INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_session "
        query = query + "(user_id, session_id, item_session, artist, first_name, gender, \
                            last_name, length, \
                            level, location, song) VALUES(%s, %s, %s, \
                                                            %s, %s, %s, \
                                                            %s, %s, %s, \
                                                            %s, %s)"
        ## Insert each columns into the table.
        session.execute(query, (int(line[10]), int(line[3]), int(line[8]), line[0], line[1], line[2], line[4], float(line[5]), \
                                line[6], line[7], line[9]))

In [ ]:
## Adding SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, first_name, last_name FROM song_playlist_session WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)

### The Third Query

In [ ]:
### For this query song, user_id are considered as the COMPOSITE KEY. \
### Because we are quering the user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [ ]:
# Create the table `user_songs_details`
query = "CREATE TABLE IF NOT EXISTS user_songs_details"
try:
    session.execute(query + "(song varchar, user_id int, first_name varchar, gender varchar, item_session int, last_name varchar, \
                                location varchar, \
                                PRIMARY KEY(song, user_id))")
except Exception as e:
    print(e)


In [ ]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Adding INSERT statements into the `query` variable
        query = "INSERT INTO user_songs_details "
        query = query + "(song, user_id, first_name, gender, item_session, last_name, \
                            location) VALUES(%s, %s, %s, \
                                                        %s, %s, %s, \
                                                        %s)"
        ## Insert each columns into the table.
        session.execute(query, (line[9], int(line[10]), line[1], line[2], int(line[3]), line[4], line[7]))

In [ ]:
## Adding SELECT statement to verify the data was entered into the table
query = "SELECT first_name, last_name FROM user_songs_details WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.first_name, row.last_name)

### Drop the tables before closing out the sessions

In [ ]:
## Drop the table before closing out the sessions
queries = ["DROP TABLE IF EXISTS music_library", "DROP TABLE IF EXISTS song_playlist_session", "DROP TABLE IF EXISTS user_songs_details"]
try:
    for query in queries:
        session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()